<a href="https://colab.research.google.com/github/ccstevie/nhl_model/blob/main/NHL_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd
from datetime import date, timedelta
from functools import reduce

In [2]:
# we will use the last 30 days as our data range
today = date.today()
print("Today's date:", today)
start = today - timedelta(days=30)
print("30 days ago:", start)

Today's date: 2024-03-01
30 days ago: 2024-01-31


In [3]:
url = f"https://www.naturalstattrick.com/teamtable.php?fromseason=20232024&thruseason=20232024&stype=2&sit=5v5&score=all&rate=n&team=all&loc=B&gpf=410&fd={start}&td={today}"
req = requests.get(url)
req.status_code

200

In [4]:
df = pd.read_html(url, header=0, index_col = 0, na_values=["-"])[0]
df.head()

,Team,GP,TOI,W,L,OTL,ROW,Points,Point %,CF,...,LDSA,LDSF%,LDGF,LDGA,LDGF%,LDSH%,LDSV%,SH%,SV%,PDO
1,Ottawa Senators,11,555:31,6,4,1,5,13,0.591,547,...,116,48.44,1,5,16.67,0.92,95.69,8.95,89.45,0.984
2,Detroit Red Wings,11,549:53,7,3,1,7,15,0.682,541,...,123,50.00,5,3,62.50,4.07,97.56,11.20,90.08,1.013
3,Los Angeles Kings,12,571:35,8,4,0,7,16,0.667,579,...,114,56.32,5,6,45.45,3.40,94.74,6.71,92.25,0.990
4,Nashville Predators,11,523:28,8,3,0,8,16,0.727,564,...,123,51.38,6,6,50.00,4.62,95.12,10.36,92.50,1.029
5,San Jose Sharks,8,379:50,1,6,1,1,3,0.188,371,...,102,45.16,3,6,33.33,3.57,94.12,7.30,89.55,0.968


In [5]:
# sort by CF% GF% XGF% HDCF% SH% and obtain rankings for each team
cf = df.sort_values(by="CF%", ascending=False, ignore_index=True)
cf.index += 1
cf.reset_index(inplace=True)
cf = cf.rename(columns = {"index":"CF%"})
cf = cf[["Team", "CF%"]]
# print cf as an example
cf

,Team,CF%,CF%
0,Carolina Hurricanes,1,57.85
1,Edmonton Oilers,2,57.42
2,Vancouver Canucks,3,54.40
3,Florida Panthers,4,53.83
4,Philadelphia Flyers,5,52.57
5,Calgary Flames,6,52.19
6,New Jersey Devils,7,52.08
7,Arizona Coyotes,8,52.04
8,Nashville Predators,9,51.70
9,Tampa Bay Lightning,10,51.51


In [6]:
gf = df.sort_values(by="GF%", ascending=False, ignore_index=True)
gf.index += 1
gf.reset_index(inplace=True)
gf = gf.rename(columns = {"index":"GF%"})
gf = gf[["Team", "GF%"]]

In [7]:
xgf = df.sort_values(by="xGF%", ascending=False, ignore_index=True)
xgf.index += 1
xgf.reset_index(inplace=True)
xgf = xgf.rename(columns = {"index":"xGF%"})
xgf = xgf[["Team", "xGF%"]]

In [8]:
hdcf = df.sort_values(by="HDCF%", ascending=False, ignore_index=True)
hdcf.index += 1
hdcf.reset_index(inplace=True)
hdcf = hdcf.rename(columns = {"index":"HDCF%"})
hdcf = hdcf[["Team", "HDCF%"]]

In [9]:
sh = df.sort_values(by="SH%", ascending=False, ignore_index=True)
sh.index += 1
sh.reset_index(inplace=True)
sh = sh.rename(columns = {"index":"SH%"})
sh = sh[["Team", "SH%"]]

In [10]:
# merge all DataFrames into one
dfs = [cf, gf, xgf, hdcf, sh]
final_df = reduce(lambda  left,right: pd.merge(left,right,on=['Team'],
                                            how='outer'), dfs)
final_df

,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
0,Carolina Hurricanes,1,57.85,1,63.89,2,56.83,2,57.69,22,8.07
1,Edmonton Oilers,2,57.42,15,51.79,1,59.59,1,65.52,14,8.76
2,Vancouver Canucks,3,54.40,19,49.06,6,53.36,5,54.02,7,9.63
3,Florida Panthers,4,53.83,2,62.50,3,54.84,3,55.87,28,7.07
4,Philadelphia Flyers,5,52.57,10,53.49,7,53.16,13,51.65,12,8.95
5,Calgary Flames,6,52.19,6,54.76,11,52.00,20,48.90,9,9.31
6,New Jersey Devils,7,52.08,8,53.85,5,53.86,11,52.12,13,8.81
7,Arizona Coyotes,8,52.04,29,40.00,13,51.26,15,50.53,20,8.30
8,Nashville Predators,9,51.70,4,58.00,15,50.17,19,49.22,5,10.36
9,Tampa Bay Lightning,10,51.51,28,40.82,19,49.45,10,52.69,19,8.40


In [11]:
# ex. get a team's stats
final_df.loc[final_df["Team"] == "Toronto Maple Leafs"]

,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
18,Toronto Maple Leafs,19,49.76,5,57.89,9,52.75,4,54.31,3,10.78


In [12]:
from get_todays_games import getGames

matchups = getGames()
matchups

[('Coyotes', 'Senators'), ('Flyers', 'Capitals'), ('Devils', 'Ducks')]

In [13]:
res = pd.DataFrame()

for away, home in matchups:
    away_df = final_df[final_df["Team"].str.contains(away)]
    home_df = final_df[final_df["Team"].str.contains(home)]
    matchup_df = pd.concat([away_df, home_df], ignore_index=True)
    res = pd.concat([res, matchup_df], ignore_index=True)
res


,Team,CF%,CF%,GF%,GF%,xGF%,xGF%,HDCF%,HDCF%,SH%,SH%
0,Arizona Coyotes,8,52.04,29,40.00,13,51.26,15,50.53,20,8.30
1,Ottawa Senators,23,48.45,25,46.00,20,49.39,18,50.23,11,8.95
2,Philadelphia Flyers,5,52.57,10,53.49,7,53.16,13,51.65,12,8.95
3,Washington Capitals,15,50.76,23,48.08,16,50.10,7,53.23,2,11.01
4,New Jersey Devils,7,52.08,8,53.85,5,53.86,11,52.12,13,8.81
5,Anaheim Ducks,25,47.35,30,37.50,28,44.34,31,39.20,24,7.98


In [14]:
f = open("result.csv", 'w')

for col in res.columns.values:
    f.write(col + ",")

f.write("\n")

i = 0
for col in res.values:
    for row in col:
        f.write(str(row) + ",")
    if i % 2 == 0:
        f.write("\n")
    else:
        f.write("\n\n")
    i += 1

f.close()